# Prevendo o resultado de jogos de futebol 

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.12


In [2]:
# Importa o findspark e inicializa
import findspark
findspark.init()

In [3]:
# Imports 
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [4]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Henrique Bardella" --iversions

Author: Henrique Bardella

findspark: 2.0.1
pyspark  : 3.3.0



## Carregando os dados

In [5]:
# Criando o Spark Context
sc = SparkContext(appName = "Lab5")

In [6]:
sc.setLogLevel("ERROR")

In [7]:
# Spark Session - Usada quando se trabalha com Dataframes no Spark
spSession = SparkSession.builder.master("local").getOrCreate()

In [8]:
# Carregando os dados e gerando um RDD
dados_time_futebol = sc.textFile("dados/dataset2.csv")

In [9]:
# Colocando o RDD em cache. Esse processo otimiza a performance.
dados_time_futebol.cache()

dados/dataset2.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [10]:
dados_time_futebol.count()

151

In [11]:
dados_time_futebol.take(5)

['media_faltas_sofridas,media_faltas_recebidas,media_cartoes_recebidos,media_chutes_a_gol,resultado',
 '4.8,3,1.4,0.3,vitoria',
 '5.1,3.8,1.6,0.2,vitoria',
 '4.6,3.2,1.4,0.2,vitoria',
 '5.3,3.7,1.5,0.2,vitoria']

In [12]:
# Removendo a primeira linha do arquivo (cabeçalho)
dados_time_futebol_2 = dados_time_futebol.filter(lambda x: "media_faltas_sofridas" not in x)
dados_time_futebol_2.count()

150

## Limpeza e Transformação dos Dados

In [13]:
# Separando as colunas
dados_time_futebol_3 = dados_time_futebol_2.map(lambda l: l.split(","))

In [14]:
# Mapeando as colunas
  # cada linha é gravada em um objeto do tipo linha, Objeto row do spark
dados_time_futebol_4 = dados_time_futebol_3.map(lambda p: Row(media_faltas_sofridas = float(p[0]), 
                                                              media_faltas_recebidas = float(p[1]), 
                                                              media_cartoes_recebidos = float(p[2]), 
                                                              media_chutes_a_gol = float(p[3]), 
                                                              resultado = p[4] ))

In [15]:
# Converte o RDD para DataFrame do Spark
df_time = spSession.createDataFrame(dados_time_futebol_4)
df_time.cache()

DataFrame[media_faltas_sofridas: double, media_faltas_recebidas: double, media_cartoes_recebidos: double, media_chutes_a_gol: double, resultado: string]

In [16]:
df_time.take(5)

[Row(media_faltas_sofridas=4.8, media_faltas_recebidas=3.0, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.3, resultado='vitoria'),
 Row(media_faltas_sofridas=5.1, media_faltas_recebidas=3.8, media_cartoes_recebidos=1.6, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=4.6, media_faltas_recebidas=3.2, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=5.3, media_faltas_recebidas=3.7, media_cartoes_recebidos=1.5, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=5.1, media_faltas_recebidas=3.5, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.2, resultado='vitoria')]

In [17]:
# Criando um índice numérico para a coluna de label target
stringIndexer = StringIndexer(inputCol = "resultado", outputCol = "idx_resultado")

In [18]:
# Treinando o string indexer
si_model = stringIndexer.fit(df_time)

In [19]:
# Aplicando o string indexer
df_time_final = si_model.transform(df_time)

In [20]:
df_time_final.select("resultado", "idx_resultado").distinct().collect()

[Row(resultado='derrota', idx_resultado=0.0),
 Row(resultado='vitoria', idx_resultado=2.0),
 Row(resultado='empate', idx_resultado=1.0)]

In [21]:
df_time_final.take(5)

[Row(media_faltas_sofridas=4.8, media_faltas_recebidas=3.0, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.3, resultado='vitoria', idx_resultado=2.0),
 Row(media_faltas_sofridas=5.1, media_faltas_recebidas=3.8, media_cartoes_recebidos=1.6, media_chutes_a_gol=0.2, resultado='vitoria', idx_resultado=2.0),
 Row(media_faltas_sofridas=4.6, media_faltas_recebidas=3.2, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.2, resultado='vitoria', idx_resultado=2.0),
 Row(media_faltas_sofridas=5.3, media_faltas_recebidas=3.7, media_cartoes_recebidos=1.5, media_chutes_a_gol=0.2, resultado='vitoria', idx_resultado=2.0),
 Row(media_faltas_sofridas=5.1, media_faltas_recebidas=3.5, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.2, resultado='vitoria', idx_resultado=2.0)]

## Análise Exploratória dos dados

In [22]:
# Estatística descritiva
df_time_final.describe().show()

+-------+---------------------+----------------------+-----------------------+------------------+---------+------------------+
|summary|media_faltas_sofridas|media_faltas_recebidas|media_cartoes_recebidos|media_chutes_a_gol|resultado|     idx_resultado|
+-------+---------------------+----------------------+-----------------------+------------------+---------+------------------+
|  count|                  150|                   150|                    150|               150|      150|               150|
|   mean|    5.843333333333332|    3.0573333333333337|      3.758000000000001|1.1993333333333331|     null|               1.0|
| stddev|   0.8280661279778625|   0.43586628493669793|     1.7652982332594667|0.7622376689603465|     null|0.8192319205190404|
|    min|                  4.3|                   2.0|                    1.0|               0.1|  derrota|               0.0|
|    max|                  7.9|                   4.4|                    6.9|               2.5|  vitoria|    

In [23]:
df_time_final.columns

['media_faltas_sofridas',
 'media_faltas_recebidas',
 'media_cartoes_recebidos',
 'media_chutes_a_gol',
 'resultado',
 'idx_resultado']

In [24]:
# Correlação entre as variáveis
for i in df_time_final.columns:
    if not(isinstance(df_time_final.select(i).take(1)[0][0], str)):
        print("correlação da variável idx_resultado com: ", i, df_time_final.stat.corr('idx_resultado', i))

correlação da variável idx_resultado com:  media_faltas_sofridas -0.460039156500237
correlação da variável idx_resultado com:  media_faltas_recebidas 0.6183715308237437
correlação da variável idx_resultado com:  media_cartoes_recebidos -0.6492418307641741
correlação da variável idx_resultado com:  media_chutes_a_gol -0.5803770334306263
correlação da variável idx_resultado com:  idx_resultado 1.0


## Pré-processamento dos dados

In [25]:
# Criando um LabeledPoint (target, Vector[features])
# Remove colunas não relevantes para o modelo ou com baixa correlação
def transformaVar(row) :
    obj = (row["resultado"], row["idx_resultado"], Vectors.dense([row["media_faltas_sofridas"], 
                                                                  row["media_faltas_recebidas"],
                                                                  row["media_cartoes_recebidos"], 
                                                                  row["media_chutes_a_gol"]]))
    return obj

In [26]:
# Aplica a função
df_time_final_RDD = df_time_final.rdd.map(transformaVar)

In [27]:
df_time_final_RDD.take(5)

[('vitoria', 2.0, DenseVector([4.8, 3.0, 1.4, 0.3])),
 ('vitoria', 2.0, DenseVector([5.1, 3.8, 1.6, 0.2])),
 ('vitoria', 2.0, DenseVector([4.6, 3.2, 1.4, 0.2])),
 ('vitoria', 2.0, DenseVector([5.3, 3.7, 1.5, 0.2])),
 ('vitoria', 2.0, DenseVector([5.1, 3.5, 1.4, 0.2]))]

In [28]:
# Converte o RDD para DataFrame
df_spark = spSession.createDataFrame(df_time_final_RDD,["resultado", "label", "features"])

In [29]:
df_spark.cache()

DataFrame[resultado: string, label: double, features: vector]

In [30]:
df_spark.select("resultado", "label", "features").show(10)

+---------+-----+-----------------+
|resultado|label|         features|
+---------+-----+-----------------+
|  vitoria|  2.0|[4.8,3.0,1.4,0.3]|
|  vitoria|  2.0|[5.1,3.8,1.6,0.2]|
|  vitoria|  2.0|[4.6,3.2,1.4,0.2]|
|  vitoria|  2.0|[5.3,3.7,1.5,0.2]|
|  vitoria|  2.0|[5.1,3.5,1.4,0.2]|
|  vitoria|  2.0|[4.9,3.0,1.4,0.2]|
|  vitoria|  2.0|[4.7,3.2,1.3,0.2]|
|  vitoria|  2.0|[4.6,3.1,1.5,0.2]|
|  vitoria|  2.0|[5.0,3.6,1.4,0.2]|
|  vitoria|  2.0|[5.4,3.9,1.7,0.4]|
+---------+-----+-----------------+
only showing top 10 rows



In [31]:
# Dados de Treino e de Teste
(dados_treino, dados_teste) = df_spark.randomSplit([0.7, 0.3])

In [32]:
dados_treino.count()

108

In [33]:
dados_teste.count()

42

## Machine Learning

In [35]:
# Cria o objeto
dtClassifier = DecisionTreeClassifier(maxDepth = 2, labelCol = "label", featuresCol = "features")

In [37]:
# Treina o objeto com dados para criar o modelo
modelo = dtClassifier.fit(dados_treino)

In [38]:
# Hiperparâmetro definido por padrão
modelo.numNodes

5

In [39]:
# Hiperparâmetro definido por nós
modelo.depth

2

In [40]:
# Previsões com dados de teste
previsoes = modelo.transform(dados_teste)

In [41]:
previsoes

DataFrame[resultado: string, label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [42]:
previsoes.select("resultado", "label", "prediction", "probability").collect()

[Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9487, 0.0513, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9487, 0.0513, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9487, 0.0513, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=1.0, probability=DenseVector([0.0, 1.0, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9487, 0.0513, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9487, 0.0513, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9487, 0.0513, 0.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row

In [43]:
# Avaliando a acurácia
avaliador = MulticlassClassificationEvaluator(predictionCol = "prediction", 
                                              labelCol = "label", 
                                              metricName = "accuracy")

In [44]:
avaliador.evaluate(previsoes)      

0.9047619047619048

In [45]:
# Resumindo as previsões - Confusion Matrix
previsoes.groupBy("label", "prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  0.0|       1.0|    2|
|  2.0|       2.0|   13|
|  0.0|       0.0|   11|
|  1.0|       1.0|   14|
|  1.0|       0.0|    2|
+-----+----------+-----+

